In [1]:
del feature_list, con_list, func_list, y_list, onset_bold_list, predicted_list, concated_list, final_onset_list

NameError: name 'feature_list' is not defined

In [ ]:
# confounds 불러오기
import pandas as pd
from nilearn import image

'''
'2207071', '2207141', '2207142', '2207143', '2207271', '2207272', '2207273',
          '2208041', '2208042', '2208043', '2208231', '2208232', '2208233', '2209022',
          '2209023', '2209222', '2209223', '2303101', '2303102', '2303103', '2303131',
          '2303132', '2303133', '2303161', '2303162', '2303163', '2303201', '2303202', '2303203'
'''

con_path = 'D:/fmri/3. derivatives 폴더에 넣어야 하는 것 (전처리 후 데이터, MNI 템플릿)/fmriprep/sub-ID{}/func/sub-ID{}_task-r{}_desc-confounds_timeseries.tsv'
base_path = 'D:/fmri/3. derivatives 폴더에 넣어야 하는 것 (전처리 후 데이터, MNI 템플릿)/fmriprep/sub-ID{}/func/sub-ID{}_task-r{}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'


sub_ids = ['2303163', '2303201', '2303202', '2303203']
r_nums = ['5','6','1','2']

feature_list = []

for sub_id in sub_ids:
    con_list = []
    func_list = []
    
    for r_num in r_nums:
        # confounds
        file_path = con_path.format(sub_id, sub_id, r_num)
        con = pd.read_csv(file_path, sep='\t')

        con1 = con.fillna(0)

        con_list.append(pd.DataFrame(con1))

        # bold
        func_path = base_path.format(sub_id, sub_id, r_num)
        data = image.load_img(func_path)
    
        func_list.append(data)

        # 전처리
    for i in range(len(func_list)):
        func_dc = image.clean_img(func_list[i], detrend=True, standardize=True, t_r=2,
                              confounds=con_list[i])
        feature_list.append(func_dc)
        

In [ ]:
feature_list

In [ ]:
onset_path = 'D:/fmri/3. derivatives 폴더에 넣어야 하는 것 (전처리 후 데이터, MNI 템플릿)/fmriprep/sub-ID{}/eve_files/sub-ID{}_task-r{}_events.tsv'

from scipy.interpolate import interp1d
import numpy as np

r_nums = ['5','6','1','2']


y_list = []

for sub_id in sub_ids:
    onset_list = []
    rjc_down_list = []
    original_scale = np.arange(0,550, 1)
    desired_scale = np.arange(0,550, 2)
    
    for r_num in r_nums:
        file_path = onset_path.format(sub_id, sub_id, r_num)
        onset_file = pd.read_csv(file_path, sep='\t')

        if r_num == '5' or r_num == '6':
            rjc_down = onset_file[onset_file['chosen_change_detail'] == 'rjc_down']
            onset_list.append(rjc_down)

            events = rjc_down['stim_onset'].values
            predictor_all = np.zeros(550, dtype=int)

            for i in [4,5]:
                events1 = events.astype(int) + i
                predictor_all[events1.astype(int)] = 1

            resampler = interp1d(original_scale, predictor_all)
            y = resampler(desired_scale)
            y_list.append(y)
            
        else:
            rjc_down_jpg = pd.concat([onset_list[0]['stim'], onset_list[1]['stim']], axis=0)
            rjc_1 = onset_file[onset_file['stim'].isin(rjc_down_jpg.values)]
            rjc_down_list.append(rjc_1)

            events = rjc_1['stim_onset'].values
            predictor_all = np.zeros(550, dtype=int)
            
            for i in [4,5]:
                events1 = events.astype(int) + i
                predictor_all[events1.astype(int)] = 1

            resampler = interp1d(original_scale, predictor_all)
            y = resampler(desired_scale)
            y_list.append(y)


In [ ]:
onset_list[0]

In [ ]:
rjc_down_list[0]

In [ ]:
y_list[0]

In [ ]:
len(y_list)

In [ ]:
# 참가자의 onset때 bold추출
# y값도 같이 만들어주기.
import nibabel as nib

onset_bold_list = []
predicted_list = []

for i in range(len(feature_list)):
    fdata = feature_list[i].get_fdata()
    onset_indices = np.where(y_list[i] == 1)[0]
    
    bold_data = fdata[:,:,:,onset_indices]
    img = nib.Nifti1Image(bold_data, feature_list[i].affine)
    onset_bold_list.append(img)

    print('file shape :', onset_bold_list[i].shape)

    if (i+1) % 4 == 1 or (i+1) % 4 == 2:
        predicted = [1]*img.shape[3]
        predicted_list.append(predicted)

    else:
        predicted2 = [0]*img.shape[3]
        predicted_list.append(predicted2)

In [ ]:
predicted_list

In [ ]:
# r5,r6,r1,r2 합치기
from nilearn import image

concated_list = []
chunk_size=4

for j in range(0, len(onset_bold_list), chunk_size):
    bold_concated = image.concat_imgs(onset_bold_list[j:j+4])
    print(bold_concated.shape)
    concated_list.append(bold_concated)

In [ ]:
# 메모리 위해 따로 저장
for i in range(len(concated_list)):
    output_path = f'D:/fmri/rjc_down/ID{sub_ids[i]}_rjc_down.nii.gz'
    print(f'the file name : ID{sub_ids[i]}_rjc_down')
    print(f'the file shape : {concated_list[i].shape}')
    nib.save(concated_list[i], output_path)

In [ ]:
# y 만들기..
chunk_size = 4

final_onset_list = []

for i in range(0, len(predicted_list), chunk_size):
    onset_y = np.concatenate(predicted_list[i:i+4])
    y = onset_y.ravel()
    print(y.shape)
    final_onset_list.append(y)

In [ ]:
# 따로 저장
for i in range(len(final_onset_list)):
    output_y = pd.DataFrame({'R2=1' : final_onset_list[i]})
    output_y_path = f'D:/fmri/rjc_down/y/ID{sub_ids[i]}_rjc_down_y.csv'
    
    print(f'the file name : ID{sub_ids[i]}_rjc_down_y')
    print(f'the file shape : {final_onset_list[i].shape}')
    
    output_y.to_csv(output_y_path, index=False)